In [ ]:
!pip -q uninstall -y pillow
!pip -q install "pillow<12" --upgrade

!pip -q install --upgrade diffusers transformers accelerate safetensors opencv-python matplotlib


In [ ]:

# TRACK 1: Face Inpainting Benchmark
# Models:
# 1) SD 1.5 Inpaint
# 2) SD 2 Inpaint
# 3) SDXL Inpaint
# 4) LaMa (GAN baseline)


import os, sys, subprocess, importlib
import torch, numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from google.colab import files
import cv2
from diffusers import AutoPipelineForInpainting

# ---------- Upload ----------
print("Upload TWO files: (1) injured image  (2) mask image (white=inpaint)")
up = files.upload()
uploaded = list(up.keys())
img_path, mask_path = uploaded[0], uploaded[1]

# ---------- Settings (T4 SAFE) ----------
MAX_SIDE = 512
SEED = 42
GUIDANCE = 7.0

STEPS_SD15 = 28
STEPS_SD2  = 28
STEPS_SDXL = 15   # keep LOW

PROMPT = (
    "a realistic photo of the same person, natural healed skin texture, "
    "consistent facial features, same identity, same lighting, high detail"
)
NEG_PROMPT = (
    "different person, changed identity, distorted face, extra eyes, "
    "cartoon, blur, artifacts, watermark"
)

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype  = torch.float16 if device == "cuda" else torch.float32
print("Device:", device)

# ---------- Helpers ----------
def load_rgb(p):
    return Image.open(p).convert("RGB")

def load_mask(p, size):
    m = Image.open(p).convert("L").resize(size, Image.NEAREST)
    m = ((np.array(m) > 127).astype(np.uint8) * 255)
    return Image.fromarray(m)

def resize(img, mask):
    w, h = img.size
    s = min(MAX_SIDE / max(w, h), 1.0)
    nw, nh = int(w*s)//8*8, int(h*s)//8*8
    return (
        img.resize((nw, nh), Image.LANCZOS),
        mask.resize((nw, nh), Image.NEAREST)
    )

def show(images, titles):
    plt.figure(figsize=(20,5))
    for i, im in enumerate(images):
        plt.subplot(1, len(images), i+1)
        plt.imshow(im)
        plt.axis("off")
        plt.title(titles[i], fontsize=10)
    plt.show()

def cleanup():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

# ---------- Load input ----------
image = load_rgb(img_path)
mask  = load_mask(mask_path, image.size)
image, mask = resize(image, mask)

show([image, mask.convert("RGB")], ["Input", "Mask"])

# ---------- Diffusion runner ----------
def run_diffusion(model_id, steps):
    pipe = AutoPipelineForInpainting.from_pretrained(
        model_id,
        torch_dtype=dtype,
        variant="fp16" if dtype == torch.float16 else None,
    )

    pipe.enable_attention_slicing()
    pipe.enable_model_cpu_offload()

    gen = torch.Generator(device).manual_seed(SEED)

    out = pipe(
        prompt=PROMPT,
        negative_prompt=NEG_PROMPT,
        image=image,
        mask_image=mask,
        guidance_scale=GUIDANCE,
        num_inference_steps=steps,
        generator=gen
    ).images[0]

    del pipe
    cleanup()
    return out

# ---------- LaMa runner ----------
def run_lama(img_pil, mask_pil):
    try:
        try:
            importlib.import_module("simple_lama_inpainting")
        except:
            subprocess.check_call(
                [sys.executable, "-m", "pip", "-q", "install", "simple-lama-inpainting"]
            )

        from simple_lama_inpainting import SimpleLama
        lama = SimpleLama()

        img_np = np.array(img_pil)
        mask_np = ((np.array(mask_pil) > 127).astype(np.uint8) * 255)

        out = lama(img_np, mask_np)
        return Image.fromarray(out)

    except Exception as e:
        print("LaMa failed → OpenCV fallback:", str(e)[:120])
        img_bgr = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
        mask_np = ((np.array(mask_pil) > 127).astype(np.uint8) * 255)
        out = cv2.inpaint(img_bgr, mask_np, 3, cv2.INPAINT_TELEA)
        return Image.fromarray(cv2.cvtColor(out, cv2.COLOR_BGR2RGB))

# ---------- Run models ----------
results = {}
results["Input"] = image

print("\nRunning SD 1.5...")
results["SD 1.5 Inpaint"] = run_diffusion(
    "runwayml/stable-diffusion-inpainting", STEPS_SD15
)

print("\nRunning SD 2...")
results["SD 2 Inpaint"] = run_diffusion(
    "sd2-community/stable-diffusion-2-inpainting", STEPS_SD2
)

print("\nRunning SDXL...")
results["SDXL Inpaint"] = run_diffusion(
    "diffusers/stable-diffusion-xl-1.0-inpainting-0.1", STEPS_SDXL
)

print("\nRunning LaMa (GAN baseline)...")
results["LaMa (GAN baseline)"] = run_lama(image, mask)

# ---------- Show ----------
show(list(results.values()), list(results.keys()))

# ---------- Save ----------
out_dir = "/content/track1_outputs"
os.makedirs(out_dir, exist_ok=True)

for k, v in results.items():
    v.save(f"{out_dir}/{k.lower().replace(' ','_')}.png")

print("\n✅ Saved outputs in:", out_dir)
print(os.listdir(out_dir))


In [ ]:
import shutil
from google.colab import files

shutil.make_archive(
    base_name="/content/track1_results",
    format="zip",
    root_dir="/content/track1_outputs"
)

files.download("/content/track1_results.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# STEP 1: Install
!pip install -q scikit-image pandas


In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim


In [ ]:

def load_rgb_resized(path, size):
    return np.array(
        Image.open(path).convert("RGB").resize(size, Image.BILINEAR)
    )

def load_mask(path):
    m = Image.open(path).convert("L")
    m = np.array(m)
    return (m > 127).astype(np.uint8)


In [ ]:
mask_img = Image.open(MASK).convert("L")
mask = (np.array(mask_img) > 127).astype(np.uint8)
H, W = mask.shape


In [ ]:
gt = load_rgb_resized(GT, (W, H))


In [ ]:
rows = []

for name, path in outputs.items():
    pred = load_rgb_resized(path, (W, H))

    psnr_val = psnr(
        gt[mask == 1],
        pred[mask == 1],
        data_range=255
    )

    ssim_val = ssim(
        gt, pred,
        data_range=255,
        channel_axis=2,
        mask=mask
    )

    rows.append([name, round(psnr_val,2), round(ssim_val,4)])

import pandas as pd
df = pd.DataFrame(rows, columns=["Model", "PSNR ↑", "SSIM ↑"])
df


In [ ]:
from matplotlib import pyplot as plt
_df_9['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
import matplotlib.pyplot as plt
from google.colab import files

fig, ax = plt.subplots(figsize=(7, 2.5))
ax.axis('tight')
ax.axis('off')

ax.table(
    cellText=df.values,
    colLabels=df.columns,
    loc='center',
    cellLoc='center'
)

plt.savefig("/content/metrics_table.png", dpi=300, bbox_inches="tight")
plt.show()

files.download("/content/metrics_table.png")


In [ ]:
df.plot(
    x="Model",
    y=["PSNR ↑", "SSIM ↑"],
    kind="bar",
    figsize=(8, 5)
)

plt.title("Face Inpainting Quantitative Comparison")
plt.ylabel("Score")
plt.grid(axis="y")
plt.tight_layout()
plt.savefig("/content/metrics_comparison_graph.png", dpi=300)
plt.show()

files.download("/content/metrics_comparison_graph.png")


In [ ]:
# =========================================================
# TRACK 2: DIFFUSION-BASED FACE RESTORATION (KAGGLE SAFE)
# =========================================================

import os
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from diffusers import StableDiffusionImg2ImgPipeline

# -------- DEVICE --------
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

# -------- PATHS --------
TRACK1_DIR = "/kaggle/working/track1_outputs"
TRACK2_DIR = "/kaggle/working/track2_restored_outputs"
os.makedirs(TRACK2_DIR, exist_ok=True)

# -------- LOAD IMAGES --------
images = {}
for f in os.listdir(TRACK1_DIR):
    if f.endswith(".png") and f != "input.png":
        images[f.replace(".png","")] = Image.open(
            os.path.join(TRACK1_DIR, f)
        ).convert("RGB")

# -------- LOAD PIPELINE --------
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=dtype
).to(device)

pipe.enable_attention_slicing()

# -------- PROMPTS --------
PROMPT = (
    "a realistic photo of the same person, "
    "natural healed skin, sharp facial features, "
    "same identity, high detail, realistic lighting"
)

NEG_PROMPT = (
    "different person, distorted face, blur, artifacts, "
    "extra eyes, cartoon, oversharpened"
)

# -------- RESTORE FUNCTION --------
def restore(img):
    return pipe(
        prompt=PROMPT,
        negative_prompt=NEG_PROMPT,
        image=img,
        strength=0.25,        # LOW change = restoration
        guidance_scale=7.0,
        num_inference_steps=20
    ).images[0]

# -------- RUN RESTORATION --------
restored = {}
for k, v in images.items():
    print("Restoring:", k)
    restored[k + "_restored"] = restore(v)

# -------- SHOW --------
plt.figure(figsize=(22,6))
i = 1
for k in images:
    plt.subplot(1, len(images)*2, i)
    plt.imshow(images[k]); plt.axis("off"); plt.title(k)
    i += 1
    plt.subplot(1, len(images)*2, i)
    plt.imshow(restored[k+"_restored"]); plt.axis("off")
    plt.title(k + " + Restored")
    i += 1
plt.show()

# -------- SAVE --------
for k, v in restored.items():
    v.save(os.path.join(TRACK2_DIR, k + ".png"))

print("DONE. SAVED IN:", TRACK2_DIR)
print(os.listdir(TRACK2_DIR))
